In [ ]:
生成具有不同细节级别的图片和视频说明。我们建议您从以下图片/视频提示开始，从此处迭代操作以获取更具体的说明。
图片：“您能编写关于图片的说明吗？”
视频：“您能编写这个视频所发生情况的说明吗？”

好的,我来为您整理一个更加清晰易读的对比表格:

| 特性 | Gemini 1.5 Flash/Pro | Gemini 1.0 Pro Vision |
|------|----------------------|------------------------|
| 文本上下文长度 | 100万词元(约4000页) | 16,384词元(约128页) |
| 代码处理能力 | 可处理整个代码库 | 限16,384词元 |
| 单次处理图片数量 | 最多3000张 | 最多16张 |
| 音频处理 | 支持,最长8.4小时 | 不支持 |
| 视频处理 | 支持,最长50分钟(有音频)<br>1小时(无音频)<br>每次最多10个视频 | 支持,最长2分钟<br>每次1个视频<br>忽略音频 |
| PDF处理 | 最多300页,30MB | 最多16页,30MB |
| 输出限制 | 无特别说明 | 最多2,048词元 |

主要区别:
1. Gemini 1.5 在各方面能力都大幅提升,尤其是处理长文本、大量图片和长视频的能力。
2. Gemini 1.5 新增了音频处理能力。
3. Gemini 1.5 在处理PDF时可以处理更多页面。
4. Gemini 1.0 Pro Vision在输出长度上有明确限制,而1.5版本没有提到此限制。

这些差异使得Gemini 1.5更适合处理大规模、复杂的多模态任务。

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240923152248.png)

Gemini API 支持将媒体文件与提示输入分开上传，这样可以在多个请求和提示中重复使用媒体文件。

其文件 API 允许每个项目存储最多 20GB 的文件，每个文件的大小不超过 2GB。文件存储 48 小时，并且可以在此期间使用您的 API 密钥进行访问，服务为免费提供。

我们将要学习如何使用文件的API，主要包括以下方法：

- **media.upload**：上传文件，创建新的 `File` 资源。
- **files.get**：获取指定文件的元数据。
- **files.list**：列出当前项目中的所有文件。
- **files.delete**：删除指定的文件。

接下来，我们将逐一详细介绍每个方法的使用方法和注意事项。


1. 上传文件——方法：upload


**端点**：

- 上传 URI（用于媒体上传请求）：

  ```
  POST https://generativelanguage.googleapis.com/upload/v1beta/files
  ```

**请求参数**：

- **file**：要上传的文件内容。
- **metadata（可选）**：文件的元数据，例如显示名称等。


**支持的文件类型**：图像、音频、文本、视频、PDF 等多种格式。
- **文件大小限制**：请参考官方文档，了解不同文件类型的大小限制。
- **重复使用**：上传的文件可以在多个请求和提示中重复使用，提高效率。

In [5]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"], transport='grpc')

model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827',safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    })


因为要参数离不开要上传的文件路径，所以可以先看看当前路径

In [6]:
import os

print(os.getcwd())

D:\PycharmProject\Gemini-系列


In [6]:
# 传入的图片文件的路径
myfile = genai.upload_file(path="images/压缩_手写照片.jpg", display_name="Sample drawing")
print(f"上传的文件信息：{myfile}")



TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

In [ ]:
# Wait for the file to finish processing
while file.state.name == "PROCESSING":
    print('Waiting for video to be processed.')
    time.sleep(2)
    video_file = genai.get_file(file.name)

In [ ]:
# 使用上传的文件进行内容生成
model = genai.GenerativeModel("gemini-1.5-flash")
result = model.generate_content([
    myfile,
    "\n\n",
    "你能描述一下这张照片吗?"
])
print(f"生成的文本内容：{result.text}")


2. 获取文件——方法：files.get

**端点**：

```
GET https://generativelanguage.googleapis.com/v1beta/{name=files/*}
```

**路径参数**：

- **name**：必需。文件的唯一名称，格式为 `files/{file_id}`。

**文件状态**：如果文件正在处理（`PROCESSING`），某些元数据可能尚不可用。


In [3]:
# 假设之前上传了一个文件并获得了其名称
myfile = genai.upload_file('media/poem.txt')
file_name = myfile.name
print(f"文件名称：{file_name}")  # 输出类似于 "files/abc-123"

# 获取文件元数据
myfile_info = genai.get_file(file_name)
print(f"文件元数据：{myfile_info}")


KeyboardInterrupt: 

3. 列出文件——方法：files.list

**端点**：

```
GET https://generativelanguage.googleapis.com/v1beta/files
```

**查询参数**：

- **pageSize（可选）**：每页返回的文件数量，默认值为 10，最大值为 100。
- **pageToken（可选）**：用于分页的令牌。

- **设置分页**：可使用 `page_size` 参数控制每页的文件数量。
- **遍历文件**：`list_files` 方法返回一个可迭代的对象，包含所有文件的 `File` 对象。

### 注意事项

- **分页处理**：如果文件数量较多，需处理分页逻辑，使用 `next_page_token` 获取下一页数据。
- **性能考虑**：合理设置 `page_size`，避免一次性请求过多数据。


In [3]:
print("我的文件列表：")
for f in genai.list_files():
    print("  ", f.name)


我的文件列表：


4. 删除文件——方法：files.delete

**端点**：

```
DELETE https://generativelanguage.googleapis.com/v1beta/{name=files/*}
```

**路径参数**：

- **name**：必需。要删除的文件的名称，格式为 `files/{file_id}`。


**删除文件**：使用 `delete_file` 方法，传入文件名称。
- **错误处理**：删除后尝试使用该文件，将引发异常，需进行错误处理。

### 注意事项

- **不可恢复**：删除操作是不可逆的，删除后无法恢复文件。
- **权限**：确保您有权限删除该文件。
- **使用检查**：在删除前，确认该文件未在其他地方被使用。

In [ ]:
# 上传文件
myfile = genai.upload_file('media/poem.txt')

# 删除文件
myfile.delete()
print(f"文件 {myfile.name} 已删除。")

# 尝试使用已删除的文件将会引发错误
try:
    model = genai.GenerativeModel("gemini-1.5-flash")
    result = model.generate_content([myfile, "Describe this file."])
except Exception as e:
    print("错误：无法使用已删除的文件。")


## 文件资源深入解析

### 重要字段解释

- **name**：文件的唯一标识符，格式为 `files/{file_id}`。
- **displayName**：文件的显示名称，便于识别。
- **mimeType**：文件的 MIME 类型，例如 `image/jpeg`。
- **sizeBytes**：文件大小，单位为字节。
- **createTime**：文件的创建时间戳。
- **updateTime**：文件的最后更新时间戳。
- **expirationTime**：文件的过期时间戳（如果适用）。
- **sha256Hash**：文件内容的 SHA-256 哈希值，用于验证完整性。
- **uri**：文件的 URI，可用于直接访问文件。
- **state**：文件的处理状态。

### 文件状态详解

- **PROCESSING**：文件正在处理，尚不可用。
- **ACTIVE**：文件已处理完成，可以使用。
- **FAILED**：文件处理失败，需检查 `error` 字段获取错误信息。

## 附录：常见问题解答

### 1. **如何处理文件上传失败的问题？**

- **检查文件格式和大小**：确保文件类型受支持，且大小不超过限制。
- **查看错误信息**：如果上传失败，`File` 对象的 `error` 字段将包含详细的错误信息。

### 2. **文件的有效期是多久？**

- **默认有效期**：文件可能有默认的过期时间，具体取决于 API 的配置。
- **查看 `expirationTime`**：通过文件的 `expirationTime` 字段，查看文件的过期时间。

### 3. **是否可以更新已上传文件的内容？**

- **不支持直接更新**：一旦文件上传，无法修改其内容。
- **解决方案**：如果需要更新文件，建议删除旧文件，重新上传新的文件。